In [1]:
import os
import re
import pandas as pd
import numpy as np
import toml
import warnings
warnings.filterwarnings("ignore")

from icecream import ic
from scipy import stats

In [ ]:
# Load the data
rtni = pd.read_csv('4RTNI_DATA.csv')

In [ ]:
len(set(rtni['SUBID']))

124

In [ ]:
# Convert the labels to the required format
def rtni_labels(row):
    if row['CDR_0_CDRTOT'] == 0:
        row['NC'] = 1
    else:
        row['NC'] = 0
    if (row['CDR_0_CDRTOT'] >= 0.5) & (row['FAQ_0_FAQTOT'] < 9):
        row['MCI'] = 1
    else:
        row['MCI'] = 0
    if (row['CDR_0_CDRTOT'] >= 1.0) & (row['FAQ_0_FAQTOT'] >= 9):
        if row['DX'] == 'PSP' or row['DX'] == 'CBS':
            row['DE'] = 1
            row['FTD'] = 1
        else:
            row['DE'] = np.NaN
            row['FTD'] = np.NaN
    else:
        row['DE'] = 0
        row['FTD'] = 0
    return row

# Extract the ID
def extract_rtni_id(row):
    return '4RTNI_' + row['SUBID']

# Convert the features to the required format
def data_convert_4rtni(df):
    if 'ID' not in df.columns:
        df['ID'] = np.NaN
    df['ID'] = df.apply(extract_rtni_id, axis=1)
    df['his_SEX'] = df['SEX'].replace({-4: np.NaN, '-4':np.NaN}).replace({'F': 'female', 'M': 'male'})
    df['his_NACCAGE'] = df['AGE_AT_TP0'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['his_EDUC'] = df['EDUCATION'].replace({-4: np.NaN, '-4':np.NaN}).replace({99.0: np.NaN}).astype(float)
    df['his_NACCNIHR'] = df['RACE'].replace({-4: np.NaN, '-4':np.NaN}).replace({1:'whi', 2:'blk', 3:'asi', 4:'haw', 5:'mul', 6:np.NaN})
    df['his_HISPANIC'] = df['LATINO'].replace({-4: np.NaN, '-4':np.NaN}).replace({0: 'no', 1: 'yes', 2:np.NaN})
    df['bat_NACCMOCA'] = df['MOCA_0_MOCATOTWITHEDUC'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['bat_NACCMMSE'] = df['MMSE_0_MMSETOT'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['bat_TRAILA'] = df['UDSTRAILS_0_TRAILA'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['bat_TRAILALI'] = df['UDSTRAILS_0_TRAILALI'].replace({-4: np.NaN, '-4':np.NaN, 71:np.NaN, 93:np.NaN}).astype(float)
    df['bat_TRAILB'] = df['UDSTRAILS_0_TRAILB'].replace({-4: np.NaN, '-4':np.NaN, 995: np.NaN}).astype(float)
    df['bat_TRAILBLI'] = df['UDSTRAILS_0_TRAILBLI'].replace({-4: np.NaN, '-4':np.NaN, 88:np.NaN, 300:np.NaN}).astype(float)
    df['gds_NACCGDS'] = df['GDS_0_GDS15TO'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['updrs_PDNORMAL'] = df['UPDRS_0_PDNORMAL'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['cdr_CDRGLOB'] = df['CDR_0_CDRTOT'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['cdr_CDRSUM'] = df['CDR_0_BOXSCORE'].replace({-4: np.NaN, '-4':np.NaN}).astype(float)
    df['npiq_DEL'] = np.where(df['NPI_Q_0_DELUSN'] == 2, 0, df['NPI_Q_0_DELSEV']).astype(float)
    df['npiq_HALL'] = np.where(df['NPI_Q_0_HLCNTNS'] == 2, 0, df['NPI_Q_0_HALSEV']).astype(float)
    df['npiq_AGIT'] = np.where(df['NPI_Q_0_AGITATE'] == 2, 0, df['NPI_Q_0_AGSEV']).astype(float)
    df['npiq_DEPD'] = np.where(df['NPI_Q_0_DPRSSN'] == 2, 0, df['NPI_Q_0_DEPSEV']).astype(float)
    df['npiq_ANX'] = np.where(df['NPI_Q_0_ANXIETY'] == 2, 0, df['NPI_Q_0_ANXSEV']).astype(float)
    df['npiq_ELAT'] = np.where(df['NPI_Q_0_EUPHORIA'] == 2, 0, df['NPI_Q_0_EUPSEV']).astype(float)
    df['npiq_APA'] = np.where(df['NPI_Q_0_APATHY'] == 2, 0, df['NPI_Q_0_APTHSEV']).astype(float)
    df['npiq_DISN'] = np.where(df['NPI_Q_0_DISINHIBITION'] == 2, 0, df['NPI_Q_0_DISSEV']).astype(float)
    df['npiq_IRR'] = np.where(df['NPI_Q_0_IRRITBLE'] == 2, 0, df['NPI_Q_0_IRRSEV']).astype(float)
    df['npiq_MOT'] = np.where(df['NPI_Q_0_MOTOR'] == 2, 0, df['NPI_Q_0_MOTSEV']).astype(float)
    df['npiq_NITE'] = np.where(df['NPI_Q_0_SLEEP'] == 2, 0, df['NPI_Q_0_SLESEV']).astype(float)
    df['npiq_APP'] = np.where(df['NPI_Q_0_EAT'] == 2, 0, df['NPI_Q_0_EATSEV']).astype(float)
    df = df.apply(rtni_labels, axis=1)

    return df
    

In [ ]:
rtni = data_convert_4rtni(rtni)[['ID', 'NC', 'MCI', 'DE', 'FTD', 'his_SEX', 'his_NACCAGE', 'his_EDUC', 'his_NACCNIHR', 'his_HISPANIC', 'bat_NACCMOCA', 'bat_NACCMMSE', 'bat_TRAILA', 'bat_TRAILALI', 'bat_TRAILB', 'bat_TRAILBLI', 'gds_NACCGDS', 'updrs_PDNORMAL', 'cdr_CDRGLOB', 'cdr_CDRSUM', 'npiq_DEL', 'npiq_HALL', 'npiq_AGIT', 'npiq_DEPD', 'npiq_ANX', 'npiq_ELAT', 'npiq_APA', 'npiq_DISN', 'npiq_IRR', 'npiq_MOT', 'npiq_NITE', 'npiq_APP']]

In [ ]:
rtni.to_csv('processed_4RTNI_DATA.csv', index=False)